<a href="https://colab.research.google.com/github/JayKarhade/MI_VCG_DL/blob/main/MI_Localization_7_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os 
import glob 
import PIL
import sklearn
import numpy as np
import pandas as pd
import scipy.io as sio 
import matplotlib.pyplot as plt
import warnings 
import cv2
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Sequential,Model
from keras.layers import Input, Dense, Activation, Flatten, Dropout,BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import SGD,Adam
from keras.utils import np_utils
from sklearn.utils import shuffle
from PIL import Image
from tensorflow.keras.regularizers import l2
#from keras.utils import to_categorical
from scipy.io import loadmat
from keras import layers
import keras
from keras import backend as K
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

In [ ]:
'''
path1 = "/content/drive/MyDrive/localization_mi/anterior_mi"
path2 = "/content/drive/MyDrive/localization_mi/anterolateral_mi"
path3 = "/content/drive/MyDrive/localization_mi/anteroseptal_mi"
path4 = "/content/drive/MyDrive/localization_mi/inferior_mi"
path5 = "/content/drive/MyDrive/localization_mi/inferolateral_mi"
path6 = "/content/drive/MyDrive/localization_mi/infero_postero_lateral_mi"
path7 = "/content/drive/MyDrive/MI_Files/myocardial_infarction"

my_list_class1 = os.listdir(path1)
my_list_class2 = os.listdir(path2)
my_list_class3 = os.listdir(path3)
my_list_class4 = os.listdir(path4)
my_list_class5 = os.listdir(path5)
my_list_class6 = os.listdir(path6)
my_list_class7 = os.listdir(path7)

data_load = []
sum = 0
y = []

instance_counter = []
curr_count = 0

for i in range(len(my_list_class1)):
  address = path1+'/'+my_list_class1[i]
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  curr_count = curr_count + temp_data.shape[0]
  instance_counter.append(curr_count)
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(0)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done

for i in range(len(my_list_class2)):
  address = path2+'/'+my_list_class2[i]
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  curr_count = curr_count + temp_data.shape[0]
  instance_counter.append(curr_count)
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(1)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done

for i in range(len(my_list_class3)):
  address = path3+'/'+my_list_class3[i]
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  curr_count = curr_count + temp_data.shape[0]
  instance_counter.append(curr_count)
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(2)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done

for i in range(len(my_list_class4)):
  address = path4+'/'+my_list_class4[i]
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  curr_count = curr_count + temp_data.shape[0]
  instance_counter.append(curr_count)
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(3)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done

for i in range(len(my_list_class5)):
  address = path5+'/'+my_list_class5[i]
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  curr_count = curr_count + temp_data.shape[0]
  instance_counter.append(curr_count)
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(4)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done

for i in range(len(my_list_class6)):
  address = path6+'/'+my_list_class6[i]
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  curr_count = curr_count + temp_data.shape[0]
  instance_counter.append(curr_count)
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(5)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done

for i in range(21):
  address = path7+'/'+my_list_class7[i]+'/matlab.mat'
  temp_data = np.asarray(loadmat(address)['chm'].T)##Get each file content
  curr_count = curr_count + temp_data.shape[0]
  instance_counter.append(curr_count)
  for j in range(temp_data.shape[0]):
    data_load.append(temp_data[j])##Append all instances in each .mat file to data_load
    y.append(6)##Class Label
  sum = sum + temp_data.shape[0]##Debugging step to make sure all instances are done


data_load = np.asarray(data_load)
y = np.asarray(y)
print(y.shape)
print(data_load.shape)

dataset = np.zeros([17338,15,651])

for i in range(17338):
  for j in range(3):
    for k in range(5):
      dataset[i,(5*j) + k] = data_load[i][j][k]

data = np.zeros([17338,651,15])
for i in range(17338):
  data[i] = dataset[i].T

np.save("/content/drive/MyDrive/MI_Files/MI_loc_label_7_classes.npy",y)
np.save("/content/drive/MyDrive/MI_Files/MI_loc_data_7_classes.npy",data)
print(instance_counter)
np.save('/content/drive/MyDrive/MI_Files/localization_instances_indices_7_classes.npy',np.asarray(instance_counter))
'''

(17338,)
(17338, 3)
[151, 313, 457, 622, 775, 888, 991, 1086, 1233, 1422, 1545, 1664, 1818, 1969, 2127, 2253, 2404, 2525, 2662, 2933, 3151, 3299, 3442, 3583, 3696, 3838, 3979, 4118, 4288, 4433, 4559, 4758, 4915, 5105, 5260, 5368, 5540, 5685, 5839, 5993, 6120, 6237, 6378, 6499, 6538, 6682, 6806, 6931, 7061, 7214, 7325, 7469, 7569, 7711, 7921, 8075, 8276, 8403, 8580, 8754, 8872, 8989, 9157, 9306, 9525, 9703, 9818, 9976, 10129, 10250, 10455, 10612, 10740, 10860, 11009, 11167, 11329, 11455, 11580, 11707, 11846, 11964, 12111, 12245, 12355, 12482, 12620, 12754, 12928, 13110, 13221, 13382, 13512, 13675, 13852, 13969, 14156, 14336, 14489, 14667, 14832, 14990, 15209, 15371, 15515, 15668, 15783, 15934, 16076, 16215, 16328, 16423, 16593, 16734, 16855, 16958, 17071, 17212, 17338]


**Run from here**

In [2]:
instance_counter = np.load('/content/drive/MyDrive/MI_Files/localization_instances_indices_7_classes.npy')
instance_counter.shape

(119,)

In [3]:
data = np.load("/content/drive/MyDrive/MI_Files/MI_loc_data_7_classes.npy")
y = np.load("/content/drive/MyDrive/MI_Files/MI_loc_label_7_classes.npy")

In [4]:
y

array([0, 0, 0, ..., 6, 6, 6])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.1, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=1/8, random_state=1)


y_tr_one_hot  = np.zeros((np.array(y_train).shape[0],7))

for i in range(np.array(y_train).shape[0]):
  label = y_train[i]
  y_tr_one_hot[i][int(label)] = 1

y_val_one_hot  = np.zeros((np.array(y_val).shape[0],7))

for i in range(np.array(y_val).shape[0]):
  label = y_val[i]
  y_val_one_hot[i][int(label)] = 1

In [5]:
##Define model
def model_define():
  model=Sequential()
  model.add(Conv1D(filters=20, kernel_size=5, activation='relu',input_shape=(651,15)))
  model.add(MaxPooling1D(pool_size=3, strides = 3))
  model.add(Conv1D(filters=60, kernel_size=5, activation='relu'))
  #model.add(Dropout(0.7))
  model.add(MaxPooling1D(pool_size=3, strides = 5))
  model.add(Conv1D(filters=120, kernel_size=5,))
  model.add(Conv1D(filters=120,kernel_size=7,))
  model.add(Flatten())
  model.add(Dense(2000, activation='relu'))
  model.add(Dense(700, activation='relu'))
  model.add(Dense(50,activation='relu'))
  model.add(Dense(7,activation='sigmoid'))
  return model

k_sum = 0
f1_sum = 0
acc_sum = 0
sens_sum = 0
spec_sum = 0

for i in range(5):
  x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.1, random_state=1)
  x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=1/8, random_state=1)


  y_tr_one_hot  = np.zeros((np.array(y_train).shape[0],7))

  for i in range(np.array(y_train).shape[0]):
    label = y_train[i]
    y_tr_one_hot[i][int(label)] = 1

  y_val_one_hot  = np.zeros((np.array(y_val).shape[0],7))

  for i in range(np.array(y_val).shape[0]):
    label = y_val[i]
    y_val_one_hot[i][int(label)] = 1
    
  print('iteration number',i)
  K.clear_session()
  modelf = model_define()

  #print(modelf.summary())
  optimizer = keras.optimizers.Adam(lr=0.00004)
  modelf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  modelf.fit(np.array(x_train), y_tr_one_hot, validation_data=(np.array(x_val), y_val_one_hot), epochs=15, batch_size=1024)

  y_te_one_hot  = np.zeros((np.array(y_test).shape[0],7))

  for i in range(np.array(y_test).shape[0]):
    label = y_test[i]
    y_te_one_hot[i][int(label)] = 1

  test_loss, test_acc = modelf.evaluate(np.array(x_test), np.array(y_te_one_hot), verbose=0)
  print('Accuracy:',test_acc)
  acc_sum = acc_sum + test_acc

  ##Evaluating Sensitivity, Accuracy and Kappa scores

  Y_pred = modelf.predict_classes(x_test)
  cm1 = confusion_matrix(y_test,Y_pred)
  print("confusion matrix \n",cm1)
  K_cappa = sklearn.metrics.cohen_kappa_score(y_test,Y_pred)
  k_sum = k_sum+K_cappa
  print("cohen kappa scores:" ,K_cappa)

  F1 = sklearn.metrics.f1_score(y_test,Y_pred,average="micro")
  f1_sum = f1_sum + F1
  print("F1 scores:", F1)

  print(classification_report(y_test, Y_pred))

iteration number 1950


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/15
14/14 [==============================] - 48s 273ms/step - loss: 1.8164 - accuracy: 0.2585 - val_loss: 1.3061 - val_accuracy: 0.4880
Epoch 2/15
14/14 [==============================] - 2s 125ms/step - loss: 1.1439 - accuracy: 0.5480 - val_loss: 0.7778 - val_accuracy: 0.6715
Epoch 3/15
14/14 [==============================] - 2s 121ms/step - loss: 0.6983 - accuracy: 0.7050 - val_loss: 0.5328 - val_accuracy: 0.7545
Epoch 4/15
14/14 [==============================] - 2s 124ms/step - loss: 0.5017 - accuracy: 0.7694 - val_loss: 0.4012 - val_accuracy: 0.8037
Epoch 5/15
14/14 [==============================] - 2s 123ms/step - loss: 0.3666 - accuracy: 0.8078 - val_loss: 0.3461 - val_accuracy: 0.8016
Epoch 6/15
14/14 [==============================] - 2s 124ms/step - loss: 0.3280 - accuracy: 0.8109 - val_loss: 0.3174 - val_accuracy: 0.8011
Epoch 7/15
14/14 [==============================] - 2s 123ms/step - loss: 0.2970 - accuracy: 0.8237 - val_loss: 0.3221 - val_accuracy: 0.8032
Epoch

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


confusion matrix 
 [[147   0   0   0   0   0  15]
 [  0 184   0   0   1   0   0]
 [  0   0 274   0   0   0  42]
 [  0   0   0 272   1   0   0]
 [  0   0   0   0 269   0  22]
 [  0   0   0   0   0 218   0]
 [ 93   0  80   0  69   0  47]]
cohen kappa scores: 0.7816306526961937
F1 scores: 0.8137254901960784
              precision    recall  f1-score   support

           0       0.61      0.91      0.73       162
           1       1.00      0.99      1.00       185
           2       0.77      0.87      0.82       316
           3       1.00      1.00      1.00       273
           4       0.79      0.92      0.85       291
           5       1.00      1.00      1.00       218
           6       0.37      0.16      0.23       289

    accuracy                           0.81      1734
   macro avg       0.79      0.84      0.80      1734
weighted avg       0.78      0.81      0.79      1734

iteration number 1950
Epoch 1/15
14/14 [==============================] - 4s 171ms/step - loss: 1

In [6]:
print("Average holdout accuracy score :",acc_sum/5)
print("Average holdout kappa score :",k_sum/5)
print("Average holdout f1 score :",f1_sum/5)
print("Average holdout sensitivity score :",sens_sum/5)
print("Average holdout specificity score :",spec_sum/5)

Average holdout accuracy score : 0.8162629842758179
Average holdout kappa score : 0.7775918375418958
Average holdout f1 score : 0.8107266435986158
Average holdout sensitivity score : 0.0
Average holdout specificity score : 0.0


**K fold Cross Validation**

In [ ]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data_cv = data[indices]
y_cv = y[indices]

In [ ]:
y_one_hot  = np.zeros((np.array(y_cv).shape[0],6))

for i in range(np.array(y_cv).shape[0]):
  label = y_cv[i]
  y_one_hot[i][int(label)] = 1

In [ ]:
##Cross Validation
folds=10
k_cv_sum = 0
f1_cv_sum = 0
acc_cv_sum = 0
sens_cv_sum = 0
spec_cv_sum = 0

kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=1)
#cvscores = []

for train,test in kfold.split(data_cv,y_cv):
  model=Sequential()
  model.add(Conv1D(filters=20, kernel_size=5, activation='relu',input_shape=(651,15)))
  model.add(MaxPooling1D(pool_size=3, strides = 3))
  model.add(Conv1D(filters=60, kernel_size=5, activation='relu'))
  #model.add(Dropout(0.7))
  model.add(MaxPooling1D(pool_size=3, strides = 5))
  model.add(Conv1D(filters=120, kernel_size=5,))
  model.add(Conv1D(filters=120,kernel_size=7,))
  model.add(Flatten())
  model.add(Dense(2000, activation='relu'))
  model.add(Dense(700, activation='relu'))
  model.add(Dense(50,activation='relu'))
  model.add(Dense(6,activation='sigmoid'))
  
  opt = keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  #Train
  model.fit(x_cv[train], y_one_hot[train],batch_size=256,epochs=15,verbose=1)
  #Test
  test_loss, test_acc = model.evaluate(data_cv[test],y_one_hot[test],batch_size=8, verbose=1)
  #Metrics
  print('Accuracy:',test_acc)
  acc_cv_sum = acc_cv_sum + test_acc
  Y_pred = model.predict_classes(data_cv[test])

  K_cappa = sklearn.metrics.cohen_kappa_score(y_cv[test],Y_pred)
  k_cv_sum = k_cv_sum+K_cappa
  print("cohen kappa scores:" ,K_cappa)

  F1 = sklearn.metrics.f1_score(y_cv[test],Y_pred,average="micro")
  f1_cv_sum = f1_cv_sum + F1
  print("F1 scores:", F1)

  cm1 = confusion_matrix(y_cv[test],Y_pred)
  print("confusion matrix \n",cm1)

  sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  sens_cv_sum = sens_cv_sum + sensitivity1
  print('Sensitivity : ', sensitivity1 )

  specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  spec_cv_sum = spec_cv_sum + specificity1
  print('Specificity : ', specificity1)

  #cvscores.append(test_acc)
  K.clear_session()

In [ ]:
print("Average 10 cv  accuracy score :",acc_cv_sum/10)
print("Average 10 cv  kappa score :",k_cv_sum/10)
print("Average 10 cv  f1 score :",f1_cv_sum/10)
print("Average 10 cv  sensitivity score :",sens_cv_sum/10)
print("Average 10 cv  specificity score :",spec_cv_sum/10)